In [59]:
!pip3 install utils
!pip3 install Pillow

In [72]:
from keras.layers import Input, Lambda, Conv2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import concatenate
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from PIL import Image, ImageFilter
# from utils import *
from tensorflow.keras.optimizers import Adam
from keras.metrics import mse
from keras.backend import set_session
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

In [73]:
def preprocess(img):
    img_cropped = img_to_array(img)
    return (np.expand_dims(img_cropped, axis=0).astype('float32'))/255


def postprocess(img):
    if len(img.shape) != 2:
      img = tf.keras.utils.array_to_img(img)
      return img
    else:
      img = np.expand_dims(img, axis=-1)
      img = tf.keras.utils.array_to_img(img)
      return img


def crop_image(img, d=32):
    x = img.size[0] - img.size[0] % d
    y = img.size[1] - img.size[1] % d

    dim = []
    dim.append(int((img.size[0] - x) / 2))
    dim.append(int((img.size[1] - y) / 2))
    dim.append(int((img.size[0] + x) / 2))
    dim.append(int((img.size[1] + y) / 2))

    img_cropped = img.crop(dim)
    return img_cropped

   
def lanczos2_kernel(factor):
    a = 2
    phase = 0.5
    kernel_size = 4 * factor + 2

    if phase == 0.5:
        kernel = np.zeros([kernel_size - 2, kernel_size - 2])

    size1 = kernel.shape[0] + 1
    size2 = kernel.shape[1] + 1
    center = kernel_size  / 2.

    for i in range(1, size1):
        for j in range(1, size2):
            if phase != 0.5:
                di = abs(i - center) / factor
                dj = abs(j - center) / factor
            else:
                di = abs(i + 0.5 - center) / factor
                dj = abs(j + 0.5 - center) / factor

            val = 1
            if di != 0:
                val = np.sin(np.pi * di / a) * np.sin(np.pi * di) * val
                val = (val * a) / (np.pi * np.pi * di * di)

            if dj != 0:
                val = np.sin(np.pi * dj / a) * np.sin(np.pi * dj) * val
                val = (val * a) / (np.pi * np.pi * dj * dj)

            kernel[i - 1][j - 1] = val

    kernel = kernel / kernel.sum()

    return kernel



def Lanczos2Conv2D(x, channel, factor=4, name=None):
    kernel = lanczos2_kernel(factor)
    weights = np.zeros((kernel.shape[0], kernel.shape[1], channel, channel))
    for i in range(channel):
        weights[:, :, i, i] = kernel
    pad_size = int((kernel.shape[0] - 1) / 2.)
    x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
    downsampling = Conv2D(channel, kernel.shape[0], strides=factor, use_bias=False)
    x = downsampling(x)
    downsampling.set_weights([weights])
    downsampling.trainable = False
    return x

In [74]:
class Hourglass:
    def __init__(self, input_size, input_channel, output_channel, nu,
                  nd, ns, ku,kd, ks,upsample='nearest', use_bias=True):
        self.input_size = input_size
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.nu = nu
        self.nd = nd
        self.ns = ns
        self.ku = ku
        self.kd = kd
        self.ks = ks
        self.upsample = upsample
        self.use_bias = True

    def upsamples(self, x, size=2):
        index = x.shape
        new_w = int(round(index[1] * size))
        new_h = int(round(index[2] * size))
        return tf.image.resize(x, [new_w, new_h], method=tf.image.ResizeMethod.BILINEAR);

    def down(self, x, n, k, use_bias=True):
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=2, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        return x

    def skip(self, x, n, k, use_bias=True):
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        return x

    def up(self, x, n, k, upsample_mode, use_bias=True):
        x = BatchNormalization()(x)
        pad_size = int((k - 1) / 2)
        x = tf.pad(x, paddings=[[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], mode='REFLECT')
        x = Conv2D(n, k, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = Conv2D(n, 1, strides=1, use_bias=use_bias)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

        x = self.upsamples(x, 2)
        return x

    def model_structure(self):
        w, h = self.input_size
        input = Input((w, h, self.input_channel))
        x = input
        n = 5
        skips = []
        for i in range(n):
            x = self.down(x, self.nd[i], self.kd[i], self.use_bias)
            if self.ns[i] != 0:
                skip = self.skip(x, self.ns[i], self.ks[i], self.use_bias)
                skips.append(skip)

        x = self.up(x, self.nu[n - 1], self.ku[n - 1], self.upsample, self.use_bias)
        skips = skips[::-1]
        for i in range(n - 1):
            j = (n - 1)- i 
            if i != 0:
                x = concatenate([x, skips[i+1]], axis=-1)
            x = self.up(x, self.nu[j-1], self.ku[j-1],self.upsample, self.use_bias)

        output = Conv2D(self.output_channel, 1, activation='sigmoid')(x)

        model = Model(inputs=input, outputs=output)
        return model



In [ ]:

method = 'random'
input_channel = 32
lr = 0.01

factor = 8
if factor == 4:
    num_iter = 4000
    sigma = 1. / 30
elif factor == 8:
    num_iter = 6000
    sigma = 1. / 20

image_path = 'zebra_GT.png'

### load data
hr_img = load_data(image_path)
hr_img = crop_image(hr_img)
lr_img = low_resolution(hr_img, factor)
hr_x = preprocess(hr_img)
lr_x = preprocess(lr_img)
postprocess(hr_x[0]).save('hr_ori.png')
postprocess(lr_x[0]).save('lr_ori.png')

### bicubic, sharpened bicubic and nearest
lr_img.resize(hr_img.size, Image.BICUBIC).save('hr_bicubic.png')
lr_img.resize(hr_img.size, Image.NEAREST).save('hr_nearest.png')
lr_img.resize(hr_img.size, Image.BICUBIC).filter(ImageFilter.UnsharpMask()).save( 'hr_sharpened.png')

### build code z
b, w, h, c = hr_x.shape

z = np.random.uniform(0, 0.1, size=(1,w,h,input_channel))
# z = make_noise(method, input_channel, (w, h))          

hg = Hourglass((w, h), input_channel, c,
                  nu=[128, 128, 128, 128, 128],
                  nd=[128, 128, 128, 128, 128],
                  ns=[4, 4, 4, 4, 128],
                  ku=[3, 3, 3, 3, 3],
                  kd=[3, 3, 3, 3, 3],
                  ks=[1, 1, 1, 1, 1],
                  upsample='bilinear'
                  )


g = hg.model_structure()
input = g.input
x = g.output
output = Lanczos2Conv2D(x, c, factor)
model = Model(inputs=input, outputs=output, name='g_trainer')

model.compile(optimizer=Adam(lr=lr), loss=mse)
model.summary()


In [ ]:
losses = []
for i in range(num_iter + 1):
    # loss = model.train_on_batch(add_noise(z, sigma), lr_x)

    temp = z + np.random.normal(0, sigma, size = z.shape)
    loss = model.train_on_batch(temp, lr_x)
    losses.append(loss)

    if i % 100 == 0:
        print('iter %d loss %f' % (i, loss))
        y = g.predict_on_batch(z)
        postprocess(y[0]).save('%d.png' % i)

model.save('model.h5')

del losses[-1]
plt.plot(list(range(num_iter)), losses)
plt.savefig('loss.png')

iter 0 loss 0.076034
iter 100 loss 0.014982
iter 200 loss 0.005821
iter 300 loss 0.002958
iter 400 loss 0.002259
iter 500 loss 0.001720
iter 600 loss 0.001423
iter 700 loss 0.001220
